# ta-lib-007 ( Feature‑Engineering + Sequence Models)
https://chatgpt.com/c/680d231a-52f4-800a-a105-9f5bfe49fca1

What's new:

1-

In [16]:
# IMPORTS
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import talib
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split


In [17]:
# PARAMETERS
DATA_PATH = '../datasets/XAGUSD-H1-rates.csv'   # Path to your historical Forex data CSV
MODEL_SAVE_PATH = 'lstm_forex_model.h5'
WINDOW_SIZE = 30              # Number of past candles used as input
FORECAST_HORIZON = 10         # Number of future candles to predict
BATCH_SIZE = 32
EPOCHS = 50
LEARNING_RATE = 0.001
TEST_SIZE = 0.2               # % of data for validation
TARGET_COLUMN = 'close'       # Which price to predict: 'close', 'open', etc.

In [ ]:
# Load your historical Forex data
df = pd.read_csv(DATA_PATH)

# Basic cleaning if necessary
df = df.dropna()
df.head()

In [ ]:
# Feature Engineering (TA-Lib indicators)
def add_ta_features(df):
    df['RSI'] = talib.RSI(df['close'], timeperiod=14)
    df['MACD'], df['MACD_signal'], df['MACD_hist'] = talib.MACD(df['close'])
    df['EMA_10'] = talib.EMA(df['close'], timeperiod=10)
    df['EMA_50'] = talib.EMA(df['close'], timeperiod=50)
    df['ATR'] = talib.ATR(df['high'], df['low'], df['close'], timeperiod=14)
    df['ADX'] = talib.ADX(df['high'], df['low'], df['close'], timeperiod=14)
    df = df.dropna()
    return df

df = add_ta_features(df)
df.head()


In [ ]:
# Scaling Features
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(df[['open', 'high', 'low', 'close', 'tickvol', 'RSI', 'MACD', 'EMA_10', 'EMA_50', 'ATR', 'ADX']])

def create_sequences(features, target, window_size, forecast_horizon):
    X, y = [], []
    for i in range(window_size, len(features) - forecast_horizon):
        X.append(features[i-window_size:i])
        y.append(target[i:i+forecast_horizon])
    return np.array(X), np.array(y)

# Prepare data
target_data = df[TARGET_COLUMN].values
X, y = create_sequences(scaled_features, target_data, WINDOW_SIZE, FORECAST_HORIZON)

print(f'Input shape: {X.shape}')
print(f'Target shape: {y.shape}')

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE, shuffle=False)

In [ ]:
def build_model(input_shape, forecast_horizon, learning_rate):
    model = Sequential([
        LSTM(64, return_sequences=True, input_shape=input_shape),
        Dropout(0.2),
        LSTM(32),
        Dropout(0.2),
        Dense(forecast_horizon)
    ])
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mse')
    return model

model = build_model((X_train.shape[1], X_train.shape[2]), FORECAST_HORIZON, LEARNING_RATE)
model.summary()


In [ ]:
# Training model
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    verbose=1
)

In [ ]:
# Predict
y_pred = model.predict(X_test)

# Inverse Scaling (Optional, if needed)
# Example if you had scaled target separately, here we skipped that


In [ ]:
# Plot first prediction vs true future prices
plt.figure(figsize=(12,6))
plt.plot(range(FORECAST_HORIZON), y_test[0], label='True Future')
plt.plot(range(FORECAST_HORIZON), y_pred[0], label='Predicted Future')
plt.title('Future Candle Price Prediction')
plt.xlabel('Time Step into Future')
plt.ylabel('Price')
plt.legend()
plt.show()


In [ ]:
model.save(MODEL_SAVE_PATH)
print(f"Model saved to {MODEL_SAVE_PATH}")
